## Star Trek Dialogue Analysis Proposal

### Data Sources: 
#### https://www.kaggle.com/datasets/birkoruzicka/startrekdialoguetranscripts/data
#### https://github.com/BirkoRuzicka/Star-Trek-Dialogue-Analysis/tree/main/additional_data

### Context & Goals: 
#### Analyzing the dialogue of all Star Trek series as a data analysis project provides a unique opportunity to explore the evolution of societal and technological themes over decades. By examining the language used by each character, how often language is used and by whom, etc., one can uncover insights into cultural shifts, predictive elements of future technologies, and the series' impact on shaping perspectives on diversity, ethics, and the human condition. 

### Key Data Concepts: 
#### Series and Episodes: The data is organized by different Star Trek series (e.g., TOS, TAS, TNG) and further divided into episodes. Each episode contains information about characters and their dialogues.

#### Characters: Dialogues are associated with specific characters from the Star Trek universe. The main cast for each series is defined, and dialogues spoken by characters outside the main cast are filtered out.

#### Dialogues: Dialogues are the spoken lines of characters in the Star Trek series. The length of each dialogue is calculated and stored.

#### Additional Data:  Additional data sources (e.g., 'tos_data.csv', 'tos_gender.csv') are used to enrich the dataset. This additional data includes information like season, year, title, and gender, and is merged into the main dataset.

In [126]:
import json
import pandas as pd
import os

In [127]:
path = r'C:\Users\anon\Documents\CareerFoundry\Data Analytics Immersion\6\Data\Raw Data'

In [128]:
# Reading in the whole data set as dictionary

with open(os.path.join(path, 'StarTrekDialogue.json'), 'r') as read_file:
    all_series = json.load(read_file)

In [129]:
# Define series variables and additional data sources for gender, etc. to be mapped, merged, and used to create series specific dataframes

series_dict = {
    'tos': {'data': all_series['TOS'], 'data_file': os.path.join(path, 'tos_data.csv'), 'gender_file': os.path.join(path, 'tos_gender.csv')},
    'tas': {'data': all_series['TAS'], 'data_file': os.path.join(path, 'tas_data.csv'), 'gender_file': os.path.join(path, 'tas_gender.csv')},
    'tng': {'data': all_series['TNG'], 'data_file': os.path.join(path, 'tng_data.csv'), 'gender_file': os.path.join(path, 'tng_gender.csv')},
    'ds9': {'data': all_series['DS9'], 'data_file': os.path.join(path, 'ds9_data.csv'), 'gender_file': os.path.join(path, 'ds9_gender.csv')},
    'voy': {'data': all_series['VOY'], 'data_file': os.path.join(path, 'voy_data.csv'), 'gender_file': os.path.join(path, 'voy_gender.csv')},
    'ent': {'data': all_series['ENT'], 'data_file': os.path.join(path, 'ent_data.csv'), 'gender_file': os.path.join(path, 'ent_gender.csv')},
    'dis': {'data': all_series['DIS'], 'data_file': os.path.join(path, 'dis_data.csv'), 'gender_file': os.path.join(path, 'dis_gender.csv')},
    'pic': {'data': all_series['PIC'], 'data_file': os.path.join(path, 'pic_data.csv'), 'gender_file': os.path.join(path, 'pic_gender.csv')},
}

In [130]:
# Define main cast variables

main_cast = {
    'tos': ['KIRK', 'SPOCK', 'UHURA', 'CHEKOV', 'SULU', 'CHAPEL', 'COMPUTER', 'MCKOY', 'SCOTT'],
    'tas': ['KIRK', 'SPOCK', 'UHURA', 'CHEKOV', 'SULU', 'CHAPEL', 'COMPUTER', 'MCKOY', 'SCOTT'],
    'tng': ['PICARD', 'RIKER', 'WORF', 'DATA', 'TROI', 'CRUSHER', 'TASHA', 'CHIEF', "O'BRIEN", 'GUINAN', 'LAFORGE', 'PULASKI', 'WESLEY'],
    'ds9': ['SISKO', 'ODO', 'KIRA', 'JAKE', 'QUARK', 'DAX', "O'BRIEN", 'BASHIR', 'WORF', 'EZRI'],
    'voy': ['JANEWAY', 'CHAKOTAY', 'TUVOK', 'PARIS', 'TORRES', 'KIM', 'EMH', 'NEELIX', 'KES', 'SEVEN', 'ICHEB', 'SESKA'],
    'ent': ['ARCHER', 'DEGRA', 'HOSHI', 'PHLOX', 'REED', 'SHRAN', "T'POL", 'TRAVIS', 'TUCKER'],
    'dis': ['BURNHAM', 'SARU', 'VOQ', 'TYLER', 'STAMETS', 'TILLY', 'LORCA', 'CULBER', 'PIKE', 'BOOK', 'NHAN', 'ADIRA', 'GRAY', 'GEORGIOU', 'DETMER', 'OWOSEKUN', "L'RELL", 'SAREK', 'CORNWELL', 'AIRIAM', 'SPOCK'],
    'pic': ['PICARD', 'AGNES', 'DAHJ', 'DATA', 'ELNOR', 'HUGH', 'SOJI', 'RAFFI', 'RIOS', 'NAREK', 'SEVEN', 'RIZZO']
}

In [131]:
# Helper function to filter out dialogue spoken by characters other than the main cast

def remove_secondary_cast_dialogue(df, main_cast_list):
    return df[df['Character'].isin(main_cast_list)]

In [132]:
# Helper function to merge in data (gender, etc.) from additional csv files

def merge_additional_data(df, data_file, gender_file):
    additional_data = pd.read_csv(os.path.join(path, data_file), index_col=0, delimiter=';', encoding='latin1')
    df = df.merge(additional_data, left_on='Episode', right_index=True)

    gender_mapping = pd.read_csv(os.path.join(path, gender_file), header=None, index_col=0, squeeze=True, delimiter=';').to_dict()
    df['Gender'] = df['Character'].map(gender_mapping)

    return df

In [144]:
# Create a dataframe for each series

series_dataframes = {}

def create_series_df(series_name, series_data, main_cast_list, data_file, gender_file):
    if series_data not in [tos, tas, tng, ds9, voy, ent, dis, pic, st]:
        print('Series not recognized')
        return

    # Transform into dataframe:
    series_df = pd.concat({k: pd.Series(v) for k, v in series_data.items()}).reset_index()
    series_df.columns = ['Episode', 'Character', 'Dialogue']

    # Get length of dialogue **calculates length of the list - this needs editing so that it retains the dialogue as an acutal list
    series_df['Dialogue Length'] = series_df['Dialogue'].str.len()

    # Drop dialogue not spoken by main cast **MAY OR MAY NOT USE THIS
#     series_df = remove_secondary_cast_dialogue(series_df, main_cast_list)

    # Merge columns regarding gender, etc. from additional data sources
    series_df = merge_additional_data(series_df, data_file, gender_file)

    series_df = series_df[['Episode', 'Season', 'Year', 'Title', 'Character', 'Gender', 'Dialogue', 'Dialogue Length']]

    series_dataframes[series_name] = series_df  # Store the DataFrame in the dictionary
    
#     series_df.to_csv(f'{series_name}_df_cleaned.csv')

    return series_df

In [145]:
for series_name, series_info in series_dict.items():
    create_series_df(series_name, series_info['data'], main_cast[series_name],
                     series_info['data_file'], series_info['gender_file'])

C:\Users\anon\AppData\Local\Temp\ipykernel_24960\3152761540.py:7: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  gender_mapping = pd.read_csv(os.path.join(path, gender_file), header=None, index_col=0, squeeze=True, delimiter=';').to_dict()
C:\Users\anon\AppData\Local\Temp\ipykernel_24960\3152761540.py:7: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  gender_mapping = pd.read_csv(os.path.join(path, gender_file), header=None, index_col=0, squeeze=True, delimiter=';').to_dict()
C:\Users\anon\AppData\Local\Temp\ipykernel_24960\3152761540.py:7: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  gender_mapping = pd.read_csv(os.path.join(path, gender_file), header=None, index_col=0, sque

In [152]:
tos_dataframe = series_dataframes['tos']
tas_dataframe = series_dataframes['tas']
tng_dataframe = series_dataframes['tng']
ds9_dataframe = series_dataframes['ds9']
ent_dataframe = series_dataframes['ent']
dis_dataframe = series_dataframes['dis']
pic_dataframe = series_dataframes['pic']
voy_dataframe = series_dataframes['voy']

In [186]:
pd.set_option('max_colwidth', None)
tos_dataframe.head(10)

Episode  Season  Year     Title   Character Gender                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [187]:
tos_episode_tos_000 = tos_dataframe[tos_dataframe['Episode'] == 'tos_000']

In [188]:
tos_episode_tos_000.shape

(21, 8)